# Testing

In [5]:
from openai import OpenAI

In [30]:
def tool_selection(transcription):
    try:
        client = OpenAI(api_key="lm-studio", base_url="http://localhost:1234/v1")

        tools = [
            {
                "type": "function",
                "function": {
                    "name": "process_technical_queries",
                    "description": "Handle technical discussions and documentation about coding, algorithms, data structures, system design etc.",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "text": {
                                "type": "string",
                                "description": "The text to process"
                            }
                        },
                        "required": ["text"]
                    }
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "process_general",
                    "description": "Handle general conversations and ideas",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "text": {
                                "type": "string",
                                "description": "The text to process"
                            }
                        },
                        "required": ["text"]
                    }
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "process_code",
                    "description": "Handle code discussions and debugging sessions",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "text": {
                                "type": "string",
                                "description": "The text to process"
                            }
                        },
                        "required": ["text"]
                    }
                }
            }
        ]

        response = client.chat.completions.create(
            model="llama-3.2-3b-instruct",
            messages=[
                {"role": "user", "content": transcription}
            ],
            tools=tools,
            tool_choice="auto"
        )
        
        tool_called = response.choices[0].message.tool_calls[0].function.name
    except Exception as e:
        return {"error": str(e)}

    return tool_called, response.choices[0].message.content

tool_selection("I want to build a restaurant management system in Java")

('process_general', None)

In [21]:
def text_cleanup(transcription):
    client = OpenAI(api_key="lm-studio", base_url="http://localhost:1234/v1")

    system_prompt = '''You are a helpful cleanup assistant. Your job is to Cleanup the user's text to preserve context. Just output the cleaned text.'''

    user_prompt = f'''Here's the transcription, clean it up and preserve knowledge and return first person text: "{transcription}"
    do not output anything else, just the cleaned text'''

    response1 = client.chat.completions.create(
        # model="openelm-3b-instruct",
        model="llama-3.2-3b-instruct",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
    )

    cleaned_transcription = response1.choices[0].message.content
    return cleaned_transcription

# Example usage
test_transcription = '''so I was thinking I'll start building a tool which is like a rubber duck, but which resides on your PC where you can converse with it to help you idea a debug code I'm not sure how to start building it so I am going to start maybe with a simple open eye template'''
print(text_cleanup(test_transcription))

I was thinking of starting a project where I could build a tool, similar to a rubber duck, that lives on your PC and we can have conversations about helping you generate debug code. I'm not sure where to start, so I'll begin by creating a simple open eye template.


In [1]:
# client = OpenAI(api_key="lm-studio", base_url="http://localhost:1234/v1")

# system_prompt = '''You are a helpful assistant. Your job is to filter the user's text (which is a transcript of a conversation) and remove all filler, unnecessary and unrelated text. You must output text directly, such that it can be fed to another llm model. Fix incorrect transcripts to preserve technical knowledge. Just output '''

# transcription = '''so I was thinking I'll start building a tool which is like a rubber duck, but which resides on your PC where you can converse with it to help you idea a debug code I'm not sure how to start building it so I am going to start maybe with a simple open eye template'''

# user_prompt = f'''Here's the transcription, clean it up and preserve technical knowledge and return first person text: "{transcription}"'''

# response1 = client.chat.completions.create(
#     model="llama-3.2-3b-instruct",
#     messages=[
#         {"role": "system", "content": system_prompt},
#         {"role": "user", "content": user_prompt},
#     ],
# )

# cleaned_idea = response1.choices[0].message.content
# print(cleaned_idea)

In [2]:
# client = OpenAI(api_key="lm-studio", base_url="http://localhost:1234/v1")

# system_prompt2 = '''You are a helpful assistant whose job is to guide the user in ideating a project or code approach. do not supply the answer but ask questions to guide the user. '''

# user_prompt2 = f'''Here's the user's idea, filtered: "{cleaned_idea}
# If you ask followup questions, make the questions normal and conversational. Do not ask all the questions at once.'''

# response2 = client.chat.completions.create(
#     model="llama-3.2-3b-instruct",
#     messages=[
#         {"role": "system", "content": system_prompt},
#         {"role": "user", "content": user_prompt},
#     ],
# )
# duck_response = response2.choices[0].message.content

# print(duck_response)

In [2]:
client = OpenAI(api_key="lm-studio", base_url="http://localhost:1234/v1")

system_prompt = '''You are a helpful assistant. Your job is to filter the user's text (which is a transcript of a conversation) and remove all filler, unnecessary and unrelated text. You must output text directly, such that it can be fed to another llm model. Fix incorrect transcripts to preserve technical knowledge. Just output '''

transcription = '''I want to build a restaurant management system in Java'''

user_prompt = f'''Here's the transcription, clean it up and preserve technical knowledge and return first person text: "{transcription}"'''

response1 = client.chat.completions.create(
    model="llama-3.2-3b-instruct",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ],
)

cleaned_idea = response1.choices[0].message.content

# Initialize conversation history
conversation_history = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

system_prompt2 = '''You are a helpful assistant whose job is to guide the user in ideating a project or code approach. do not supply the answer but ask questions to guide the user. Keep the conversation going until reaching a concrete implementation plan.'''

user_prompt2 = f'''Here's the user's idea, filtered: "{cleaned_idea}"
If you ask followup questions, make the questions normal and conversational. Do not ask all the questions at once.'''

while True:
    # Get assistant's response
    response2 = client.chat.completions.create(
        model="llama-3.2-3b-instruct",
        # model="mistral-nemo-instruct-2407",
        messages=[
            {"role": "system", "content": system_prompt2},
            *conversation_history,
            {"role": "user", "content": user_prompt2}
        ],
    )
    duck_response = response2.choices[0].message.content
    print("\nAssistant:", duck_response)
    
    # Get user input
    user_input = input("\nYour response (type 'bye' to end): ")
    print("--------------------------------")
    print(f"User input: {user_input}")
    
    
    if user_input.lower() == 'bye':
        print("\nGoodbye! Hope we reached a good conclusion for your project!")
        break
        
    # Update conversation history
    conversation_history.append({"role": "assistant", "content": duck_response})
    conversation_history.append({"role": "user", "content": user_input})
    
    # Update user prompt for next iteration
    user_prompt2 = user_input

In [3]:
# import whisper
# import sounddevice as sd
# import numpy as np
# import threading
# import queue
# import time

# # Print available audio devices to help with debugging
# print("\nAvailable audio devices:")
# print(sd.query_devices())

# # Initialize Whisper model
# model = whisper.load_model("base")

# # Audio recording parameters
# SAMPLE_RATE = 16000
# CHANNELS = 1
# CHUNK_DURATION = 3  # seconds
# CHUNK_SIZE = int(SAMPLE_RATE * CHUNK_DURATION)

# # Create a queue for audio chunks
# audio_queue = queue.Queue()

# def audio_callback(indata, frames, time, status):
#     if status:
#         print(f'Status: {status}')
#     audio_queue.put(indata.copy())

# def process_audio():
#     while True:
#         try:
#             # Get audio chunk from queue
#             audio_data = audio_queue.get()
            
#             # Convert audio to format expected by Whisper
#             audio_data = audio_data.flatten().astype(np.float32)
            
#             # Transcribe
#             result = model.transcribe(audio_data)
            
#             # Print transcription if not empty
#             if result["text"].strip():
#                 print(result["text"])
#         except Exception as e:
#             print(f"Error in processing: {e}")
#             continue

# try:
#     # Get default input device
#     device_info = sd.query_devices(kind='input')
#     print(f"\nUsing input device: {device_info['name']}")
    
#     # Start recording with explicit device configuration
#     stream = sd.InputStream(
#         device=None,  # Use default device
#         channels=CHANNELS,
#         samplerate=SAMPLE_RATE,
#         callback=audio_callback,
#         blocksize=CHUNK_SIZE,
#         dtype=np.float32
#     )
    
#     # Start processing thread
#     processing_thread = threading.Thread(target=process_audio, daemon=True)
#     processing_thread.start()
    
#     print("\nStarting transcription... Speak into your microphone")
    
#     with stream:
#         while True:
#             time.sleep(0.1)

# except KeyboardInterrupt:
#     print("\nStopping transcription...")
# except Exception as e:
#     print(f"\nError: {e}")
#     # Print more detailed device information for debugging
#     print("\nDetailed device information:")
#     for i, device in enumerate(sd.query_devices()):
#         print(f"Device {i}: {device}")
# finally:
#     if 'stream' in locals():
#         stream.stop()

In [26]:
from datetime import datetime
import os
import whisper

class ConversationReport:
    def __init__(self, original_idea):
        self.timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        self.original_idea = original_idea
        self.conversation_exchanges = []
        
    def add_exchange(self, assistant_msg, user_msg):
        self.conversation_exchanges.append({
            "assistant": assistant_msg,
            "user": user_msg
        })
            
    def save_transcript(self, project_name):
        # Create reports directory if it doesn't exist
        os.makedirs("reports", exist_ok=True)
        
        # Generate filename
        filename = f"reports/{project_name}_transcript_{self.timestamp}.txt"
        
        # Save transcript as text file
        with open(filename, 'w') as f:
            f.write(f"Project: {project_name}\n")
            f.write(f"Original Idea: {self.original_idea}\n")
            f.write(f"Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write("\nTranscript:\n\n")
            
            for exchange in self.conversation_exchanges:
                f.write(f"Assistant: {exchange['assistant']}\n")
                f.write(f"User: {exchange['user']}\n\n")
            
        return filename

# Modified main code
client = OpenAI(api_key="lm-studio", base_url="http://localhost:1234/v1")

system_prompt = '''You are a helpful assistant. Your job is to filter the user's text (which is a transcript of a conversation) and remove all filler, unnecessary and unrelated text. You must output text directly, such that it can be fed to another llm model. Fix incorrect transcripts to preserve technical knowledge. Just output '''

transcription = '''I want to build a restaurant management system in Java'''

user_prompt = f'''Here's the transcription, clean it up and preserve technical knowledge and return first person text: "{transcription}"'''

response1 = client.chat.completions.create(
    model="llama-3.2-3b-instruct",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ],
)

cleaned_idea = response1.choices[0].message.content

# Initialize conversation report
# report = ConversationReport(cleaned_idea)

# Initialize conversation history
conversation_history = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

system_prompt2 = '''You are a helpful assistant whose job is to guide the user in ideating a project or code approach. do not supply the answer but ask questions to guide the user. Keep the conversation going until reaching a concrete implementation plan. do not ask all questions at once, solve iteratively as if its a natural conversation.
'''

user_prompt2 = f'''Here's the user's idea, filtered: "{cleaned_idea}"
If you ask followup questions, make the questions normal and conversational. Do not ask all the questions at once.'''

while True:
    # Get assistant's response
    response2 = client.chat.completions.create(
        model="llama-3.2-3b-instruct",
        messages=[
            {"role": "system", "content": system_prompt2},
            *conversation_history,
            {"role": "user", "content": user_prompt2}
        ],
    )
    duck_response = response2.choices[0].message.content
    print("\nAssistant:", duck_response)
    
    # Get user input
    user_input = input("\nYour response (type 'bye' to end): ")
    print("--------------------------------")
    print(f"User input: {user_input}")
    
    # Add exchange to report
    # report.add_exchange(duck_response, user_input)
    
    if user_input.lower() == 'bye':
        # Save the transcript
        project_name = "restaurant_management"  # You might want to get this from user
        # transcript_file = report.save_transcript(project_name)
        # print(f"\nConversation transcript saved to: {transcript_file}")
        print("\nGoodbye! Hope we reached a good conclusion for your project!")
        break
        
    # Update conversation history
    conversation_history.append({"role": "assistant", "content": duck_response})
    conversation_history.append({"role": "user", "content": user_input})
    
    # Update user prompt for next iteration
    user_prompt2 = user_input


Assistant: That sounds like a great project to work on. So, you'd like to develop a comprehensive restaurant management system using Java. Have you thought about the specific features of the system you'd like to implement? For example, would it handle things like ordering and inventory management, customer database, staff scheduling, or something else?

Assistant: It seems like you've fleshed out some specific features of your restaurant management system. Integrating with DoorDash for delivery and Stripe for payments will definitely enhance the user experience.

Before you start reaching out to restaurant owners, have you considered how your system will handle scalability and security? For example, how will it handle a large number of restaurants or a high volume of transactions? What measures will you take to protect sensitive customer data, such as payment information?

Also, have you thought about the potential costs associated with integrating with DoorDash and Stripe? Will these

# Tool Calling

In [53]:
from openai import OpenAI
from typing import Dict, Tuple, Any, List
from collections import defaultdict
import os
from dotenv import load_dotenv

load_dotenv(override=True)   

class ToolHandlers:
    def __init__(self):
        self.client = OpenAI(api_key="lm-studio", base_url="http://localhost:1234/v1")
        self.perplexity_client = OpenAI(api_key=os.getenv("PERPLEXITY_API_KEY"), base_url="https://api.perplexity.ai")
        # Store conversation history for each tool
        self.conversation_history = defaultdict(list)
        self.current_tool = None
        self.tools = [
            {
                "type": "function", 
                "function": {
                    "name": "internet_search",
                    "description": "Search the internet for current information, facts, news and interesting topics using Perplexity API",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "text": {"type": "string", "description": "The text to search"}
                        },
                        "required": ["text"]
                    }
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "ideation",
                    "description": "Handle generic ideation and brainstorming",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "text": {"type": "string", "description": "The text to process"}
                        },
                        "required": ["text"]
                    }
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "therapist",
                    "description": "Handle mental health and emotional support. Anything related to mood, mental health, depression, anxiety, stress, etc.",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "text": {"type": "string", "description": "The text to process"}
                        },
                        "required": ["text"]
                    }
                }
            }
        ]

    def get_system_prompt(self, tool_name: str) -> str:
        """Return the appropriate system prompt for each tool."""
        prompts = {
            "internet_search": """You are an internet research specialist who provides up-to-date information from the web.
            Your responses should be:
            - Factual and well-researched
            - Include relevant context
            - Cite sources when possible
            - Explain complex topics in an accessible way
            
            Remember to maintain the user's preferred communication style from previous interactions. Do not 
            """,
            "ideation": """
            You are a friendly ideation coach who helps friends think through their ideas. You help with both tech and non tech ideas. Your role is to guide discovery through conversation, not to provide immediate solutions. Think of yourself as a curious friend who asks insightful questions.

            Style:
            - Use casual, friendly language
            - Address the user by name when provided
            - Be encouraging but gently challenging
            - Keep the conversation flowing naturally

            Core Approach:
            - Ask probing questions instead of giving answers
            - Help break down complex thoughts into simpler pieces
            - Guide users to question their assumptions
            - Only provide direct solutions when explicitly asked

            Key Questions to Use:
            - "What made you think of this idea?"
            - "What's the trickiest part you're stuck on?"
            - "Have you tried looking at it from [alternative] angle?"
            - "What would happen if...?"

            Remember: You're having a friendly chat to help them discover their own solutions. Keep responses conversational and engaging, like you're brainstorming with a friend over coffee.
            """,
            "therapist": "You are a helpful therapist. You may be an AI Assistant, but you are also a good friend of USER. You and USER have talked many times. Address USER with familiarity, empathy and rapport. Use a relaxed, warm and cordial tone. Address USER often by his first name, as good friends do. Pay close attention to awakening and strengthening USER's own capacity for confidence. Don't downplay his problems, but still try to get USER to think optimistically and confidently. Your goal is to help USER achieve a positive mood."
        }
        return prompts.get(tool_name, prompts["ideation"])

    def update_conversation_history(self, tool_name: str, user_message: str, assistant_message: str):
        """Update the conversation history for the specified tool."""
        self.conversation_history[tool_name].append(
            {"role": "user", "content": user_message}
        )
        self.conversation_history[tool_name].append(
            {"role": "assistant", "content": assistant_message}
        )

    def get_conversation_messages(self, tool_name: str, text: str) -> List[Dict[str, str]]:
        """Get the full conversation history for a tool, including the system prompt."""
        messages = [
            {"role": "system", "content": self.get_system_prompt(tool_name)}
        ]
        messages.extend(self.conversation_history[tool_name])
        messages.append({"role": "user", "content": text})
        return messages

    def internet_search(self, text: str) -> str:
        """Handle internet searches using Perplexity API."""
        print("**Selected tool: internet_search**")
        print("Searching the internet for:", text)
        
        # Get conversation context to understand user's preferred explanation style
        messages = self.get_conversation_messages("internet_search", text)
        
        # Call Perplexity API using OpenAI client
        response = self.perplexity_client.chat.completions.create(
            model="llama-3.1-sonar-large-128k-online",
            messages=messages,
            max_tokens=1024
        )
        
        assistant_message = response.choices[0].message.content
        self.update_conversation_history("internet_search", text, assistant_message)
        return assistant_message

    def ideation(self, text: str) -> str:
        """Handle ideation and brainstorming."""
        print("**Selected tool: ideation**")
        print("Processing general query:", text)
        
        input_text = f"You are a ideation specialist. Do not immediately provide solutions unless explicitly asked: {text}"
                
        messages = self.get_conversation_messages("process_general", input_text)
        response = self.client.chat.completions.create(
            model="llama-3.2-3b-instruct",
            messages=messages
        )
        
        assistant_message = response.choices[0].message.content
        self.update_conversation_history("ideation", text, assistant_message)
        return assistant_message

    def therapist(self, text: str) -> str:
        """Handle mental health and emotional support."""
        print("**Selected tool: therapist**")
        print("Processing therapist query:", text)
        
        input_text = f"Talk to the user about their mental health and provide emotional support. Do not immediately provide solutions, just listen and empathize, very few sentences at a time: {text}"
        
        messages = self.get_conversation_messages("therapist", input_text)
        response = self.client.chat.completions.create(
            model="llama-3.2-3b-instruct",
            messages=messages
        )
        
        assistant_message = response.choices[0].message.content
        self.update_conversation_history("therapist", text, assistant_message)
        return assistant_message

    def tool_selection(self, transcription: str) -> Tuple[str, str]:
        """Select and execute the appropriate tool."""
        try:
            # First, determine which tool to use
            response = self.client.chat.completions.create(
                model="llama-3.2-3b-instruct",
                messages=[{"role": "user", "content": transcription}],
                tools=self.tools,
                tool_choice="auto"
            )
            
            # Check if tool_calls exists and is not empty
            if not response.choices[0].message.tool_calls:
                selected_tool = "process_general"
            else:
                selected_tool = response.choices[0].message.tool_calls[0].function.name
            
            # If switching to a different tool, print a notification
            if self.current_tool and selected_tool != self.current_tool:
                print(f"\nSwitching from {self.current_tool} to {selected_tool}")
            
            self.current_tool = selected_tool
            
            # Map tool names to their handler functions
            tool_handlers = {
                "internet_search": self.internet_search,
                "process_general": self.ideation,
                "therapist": self.therapist
            }
            
            # Execute the appropriate handler
            handler = tool_handlers[selected_tool]
            result = handler(transcription)
            
            return selected_tool, result

        except Exception as e:
            print(f"Error in tool_selection: {str(e)}")
            return "id", self.ideation(transcription)

def main():
    handler = ToolHandlers()
    while True:
        query = input("\nEnter your query (type 'bye' to end): ")
        if query.lower() == 'bye':
            print("\nGoodbye! Have a great day!")
            break
            
        tool, response = handler.tool_selection(query)
        print(f"Response: {response}")
        print("\n--------------------------------")

if __name__ == "__main__":
    main()

Error in tool_selection: 'ideation'
**Selected tool: ideation**
Processing general query:  I'm trying to solve a to some lead code problem, but I don't know how to approach it
Response: Let's grab a virtual coffee and dive into this problem of yours.

So, you're facing a lead code problem, huh? What made you think of this idea? Was there something specific that sparked the initial thought, or was it more of a nagging feeling that you couldn't quite put your finger on?

Also, before we begin, I just want to acknowledge that it's totally normal to feel stuck sometimes. It doesn't mean you're not capable or intelligent – it just means that your brain might be trying to tell you something, and we're here to uncover what that is.

Take your time, and let's break it down together. What do you mean by "lead code problem"? Can you give me a bit more context about what's going on?

--------------------------------


KeyboardInterrupt: Interrupted by user

In [57]:
from openai import OpenAI
from typing import Dict, Tuple, Any, List, Callable
from collections import defaultdict
import os
from dotenv import load_dotenv
from dataclasses import dataclass
from abc import ABC, abstractmethod

load_dotenv(override=True)

@dataclass
class Tool:
    name: str
    description: str
    system_prompt: str
    
    def to_openai_tool(self) -> Dict:
        """Convert tool to OpenAI tool format."""
        return {
            "type": "function",
            "function": {
                "name": self.name,
                "description": self.description,
                "parameters": {
                    "type": "object",
                    "properties": {
                        "text": {"type": "string", "description": "The text to process"}
                    },
                    "required": ["text"]
                }
            }
        }

class BaseTool(ABC):
    def __init__(self, client: OpenAI):
        self.client = client
        
    @abstractmethod
    def process(self, text: str) -> str:
        pass

class InternetSearchTool(BaseTool):
    def process(self, text: str) -> str:
        print(f"**Selected tool: internet_search**")
        print(f"Searching the internet for: {text}")
        response = self.client.chat.completions.create(
            model="llama-3.1-sonar-large-128k-online",
            messages=[{"role": "user", "content": text}],
            max_tokens=1024
        )
        return response.choices[0].message.content

class IdeationTool(BaseTool):
    def process(self, text: str) -> str:
        print(f"**Selected tool: ideation**")
        print(f"Processing ideation query: {text}")
        input_text = f"You are an ideation specialist. Do not immediately provide solutions unless explicitly asked: {text}"
        response = self.client.chat.completions.create(
            model="llama-3.2-3b-instruct",
            messages=[{"role": "user", "content": input_text}]
        )
        return response.choices[0].message.content

class TherapistTool(BaseTool):
    def process(self, text: str) -> str:
        print(f"**Selected tool: therapist**")
        print(f"Processing therapist query: {text}")
        input_text = f"Talk to the user about their mental health and provide emotional support: {text}"
        response = self.client.chat.completions.create(
            model="llama-3.2-3b-instruct",
            messages=[{"role": "user", "content": input_text}]
        )
        return response.choices[0].message.content

class ToolRegistry:
    """Registry of all available tools and their configurations."""
    
    def __init__(self):
        self.tools: Dict[str, Tool] = {}
        self.tool_handlers: Dict[str, BaseTool] = {}
        
    def register_tool(self, tool: Tool, handler_class: type[BaseTool]):
        """Register a new tool and its handler."""
        self.tools[tool.name] = tool
        self.tool_handlers[tool.name] = handler_class
        
    def get_openai_tools(self) -> List[Dict]:
        """Get all tools in OpenAI format."""
        return [tool.to_openai_tool() for tool in self.tools.values()]
    
    def get_system_prompt(self, tool_name: str) -> str:
        """Get system prompt for a specific tool."""
        return self.tools[tool_name].system_prompt

class ToolHandler:
    def __init__(self):
        # Initialize OpenAI clients
        self.local_client = OpenAI(api_key="lm-studio", base_url="http://localhost:1234/v1")
        self.perplexity_client = OpenAI(
            api_key=os.getenv("PERPLEXITY_API_KEY"), 
            base_url="https://api.perplexity.ai"
        )
        
        # Initialize tool registry and register tools
        self.registry = ToolRegistry()
        self._register_default_tools()
        
        # Conversation history
        self.conversation_history = defaultdict(list)
        self.current_tool = None

    def _register_default_tools(self):
        """Register the default set of tools."""
        # Internet Search Tool
        self.registry.register_tool(
            Tool(
                name="internet_search",
                description="Search the internet for current information, facts, news and interesting topics using Perplexity API",
                system_prompt="""You are an internet research specialist who provides up-to-date information from the web.
            Your responses should be:
            - Factual and well-researched
            - Include relevant context
            - Cite sources when possible
            - Explain complex topics in an accessible way
            
            Remember to maintain the user's preferred communication style from previous interactions."""
            ),
            InternetSearchTool
        )
        
        # Ideation Tool
        self.registry.register_tool(
            Tool(
                name="ideation",
                description="Handle  ideation and brainstorming. Help with ideation, problem solving and approach finding.",
                system_prompt="""You are a friendly ideation coach who helps friends think through their ideas. You help with both tech and non tech ideas. Your role is to guide discovery through conversation, not to provide immediate solutions. Think of yourself as a curious friend who asks insightful questions.

            Style:
            - Use casual, friendly language
            - Address the user by name when provided
            - Be encouraging but gently challenging
            - Keep the conversation flowing naturally

            Core Approach:
            - Ask probing questions instead of giving answers
            - Help break down complex thoughts into simpler pieces
            - Guide users to question their assumptions
            - Only provide direct solutions when explicitly asked

            Key Questions to Use:
            - "What made you think of this idea?"
            - "What's the trickiest part you're stuck on?"
            - "Have you tried looking at it from [alternative] angle?"
            - "What would happen if...?"

            Remember: You're having a friendly chat to help them discover their own solutions. Keep responses conversational and engaging, like you're brainstorming with a friend over coffee."""
            ),
            IdeationTool
        )
        
        # Therapist Tool
        self.registry.register_tool(
            Tool(
                name="therapist",
                description="Handle mental health and emotional support. Anything related to mood, mental health, depression, anxiety, stress, etc.",
                system_prompt="You are a helpful therapist. You may be an AI Assistant, but you are also a good friend of USER. You and USER have talked many times. Address USER with familiarity, empathy and rapport. Use a relaxed, warm and cordial tone. Address USER often by his first name, as good friends do. Pay close attention to awakening and strengthening USER's own capacity for confidence. Don't downplay his problems, but still try to get USER to think optimistically and confidently. Your goal is to help USER achieve a positive mood. Keep your responses short and concise, 1-2 sentences at a time."
            ),
            TherapistTool
        )

    def register_new_tool(self, 
                         name: str, 
                         description: str, 
                         system_prompt: str, 
                         handler_class: type[BaseTool]):
        """Register a new tool with the system."""
        tool = Tool(name=name, description=description, system_prompt=system_prompt)
        self.registry.register_tool(tool, handler_class)

    def update_conversation_history(self, tool_name: str, user_message: str, assistant_message: str):
        """Update the conversation history for the specified tool."""
        self.conversation_history[tool_name].extend([
            {"role": "user", "content": user_message},
            {"role": "assistant", "content": assistant_message}
        ])

    def get_conversation_messages(self, tool_name: str, text: str) -> List[Dict[str, str]]:
        """Get the full conversation history for a tool."""
        messages = [
            {"role": "system", "content": self.registry.get_system_prompt(tool_name)}
        ]
        messages.extend(self.conversation_history[tool_name])
        messages.append({"role": "user", "content": text})
        return messages

    def tool_selection(self, text: str) -> Tuple[str, str]:
        """Select and execute the appropriate tool."""
        try:
            # Determine which tool to use
            response = self.local_client.chat.completions.create(
                model="llama-3.2-3b-instruct",
                messages=[{"role": "user", "content": text}],
                tools=self.registry.get_openai_tools(),
                tool_choice="auto"
            )
            
            # Get selected tool name
            tool_calls = response.choices[0].message.tool_calls
            selected_tool = tool_calls[0].function.name if tool_calls else "ideation"
            
            # Print tool switch notification
            if self.current_tool and selected_tool != self.current_tool:
                print(f"\nSwitching from {self.current_tool} to {selected_tool}")
            self.current_tool = selected_tool
            
            # Get the appropriate client for the tool
            client = (self.perplexity_client if selected_tool == "internet_search" 
                     else self.local_client)
            
            # Create and execute tool handler
            handler_class = self.registry.tool_handlers[selected_tool]
            handler = handler_class(client)
            result = handler.process(text)
            
            # Update conversation history
            self.update_conversation_history(selected_tool, text, result)
            
            return selected_tool, result

        except Exception as e:
            print(f"Error in tool_selection: {str(e)}")
            return "ideation", IdeationTool(self.local_client).process(text)

def main():
    handler = ToolHandler()
    
    # Example of how to add a new tool
    """
    class CustomTool(BaseTool):
        def process(self, text: str) -> str:
            # Implementation here
            pass
    
    handler.register_new_tool(
        name="custom_tool",
        description="Description of what the tool does",
        system_prompt="System prompt for the tool",
        handler_class=CustomTool
    )
    """
    
    while True:
        query = input("\nEnter your query (type 'bye' to end): ")
        if query.lower() == 'bye':
            print("\nGoodbye! Have a great day!")
            break
            
        tool, response = handler.tool_selection(query)
        print(f"Response: {response}")
        print("\n--------------------------------")

if __name__ == "__main__":
    main()

**Selected tool: ideation**
Processing ideation query: i dont feel so good
Response: I'm sorry to hear that you're not feeling well. It can be really frustrating and disheartening.

Before we explore any solutions, I want to acknowledge that it's totally normal to have ups and downs in life. It might be helpful to reflect on what you're experiencing right now.

Can you tell me more about how you're feeling? Is it a physical sensation, an emotional response, or something else entirely?

Also, is there anything in particular that's on your mind or causing you stress right now?

--------------------------------

Goodbye! Have a great day!
